In [10]:
'''
Objective:
•
Develop a forecasting model based on recorded solar power consumption data collected by city councils to provide insights into solar energy usage trends and facilitate the transition towards renewable energy sources.
Constraints:
•
Ensure accurate preprocessing of the solar power consumption dataset, handling missing values, outliers, and seasonality patterns, to prepare the data for forecasting model training while preserving data quality and integrity.
•
Employ appropriate forecasting techniques such as time series analysis, ARIMA, or machine learning algorithms to generate reliable predictions and insights into solar energy consumption patterns for informed decision-making.
Benefits/Impact:
•
The forecasting model enables city councils and businesses to anticipate future solar power consumption trends, facilitating proactive measures in energy planning, infrastructure investment, and policy formulation to promote renewable energy adoption.
•
By leveraging insights from the forecasting model, businesses and city councils can optimize resource allocation, reduce reliance on nonrenewable energy sources, mitigate environmental impacts, and foster sustainability, leading to long-term cost savings, environmental conservation, and community well-being.

'''



import pandas as pd
import numpy as np
import pickle
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error
from math import sqrt

In [4]:
solar = pd.read_csv('solarpower_cumuldaybyday2.csv')
solar

date  cum_power
0     26/10/2011        0.1
1     27/10/2011       10.2
2     28/10/2011       20.2
3     29/10/2011       29.6
4     30/10/2011       34.2
...          ...        ...
2553  22/10/2018    28101.0
2554  23/10/2018    28109.0
2555  24/10/2018    28115.0
2556  25/10/2018    28117.0
2557  26/10/2018    28120.0

[2558 rows x 2 columns]

In [42]:
#data preprocessing
solar['date'] = pd.to_datetime(solar['date'])
solar.set_index('date', inplace = True)

In [6]:
solar.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2558 entries, 0 to 2557
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   date       2558 non-null   object 
 1   cum_power  2558 non-null   float64
dtypes: float64(1), object(1)
memory usage: 40.1+ KB


In [7]:
solar.isna().sum()

date         0
cum_power    0
dtype: int64

In [47]:
from scipy.stats.mstats import winsorize
alpha = 0.05
wins = solar.apply(lambda x: winsorize(x, limits =[alpha, alpha]).flatten(), axis = 0)

In [59]:
outlier_df = pd.DataFrame(wins, index = solar.index, columns = solar.columns)
outlier_df

cum_power
date                 
2011-10-26      461.2
2011-10-27      461.2
2011-10-28      461.2
2011-10-29      461.2
2011-10-30      461.2
...               ...
2018-10-22    26044.0
2018-10-23    26044.0
2018-10-24    26044.0
2018-10-25    26044.0
2018-10-26    26044.0

[2558 rows x 1 columns]

In [60]:
outlier_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2558 entries, 2011-10-26 to 2018-10-26
Data columns (total 1 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   cum_power  2558 non-null   float64
dtypes: float64(1)
memory usage: 40.0 KB


In [61]:
train =outlier_df.head(2000)
test = outlier_df.tail(558)

In [62]:
model = ARIMA(train['cum_power'], order=(5,1,0))

C:\ProgramData\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
C:\ProgramData\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
C:\ProgramData\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)


In [63]:
model_fit = model.fit()

In [64]:
model_fit.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                               SARIMAX Results                                
==============================================================================
Dep. Variable:              cum_power   No. Observations:                 2000
Model:                 ARIMA(5, 1, 0)   Log Likelihood               -6000.022
Date:                Mon, 12 Feb 2024   AIC                          12012.044
Time:                        18:22:16   BIC                          12045.647
Sample:                    10-26-2011   HQIC                         12024.383
                         - 04-16-2017                                         
Covariance Type:                  opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1          0.4120      0.017     23.598      0.000       0.378       0.446
ar.L2          0.1358      0.018      7.722      0.000       0.101       0.170
ar.L3          0.1638      0.018      8.865      0.000       0.128       0.200
ar.L4          0.1205      0.019      6.350      0.000       0.083       0.158
ar.L5          0.1365      0.018      7.535      0.000       0.101       0.172
sigma2        23.6633      0.612     38.651      0.000      22.463      24.863
===================================================================================
Ljung-Box (L1) (Q):                   0.71   Jarque-Bera (JB):                89.14
Prob(Q):                              0.40   Prob(JB):                         0.00
Heteroskedasticity (H):               0.90   Skew:                             0.01
Prob(H) (two-sided):                  0.18   Kurtosis:                         4.03
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [65]:
forecast = model_fit.forecast(steps = len(test))

In [66]:
forecast

2017-04-17    20897.915215
2017-04-18    20912.134311
2017-04-19    20926.891587
2017-04-20    20940.624518
2017-04-21    20954.735112
                  ...     
2018-10-22    22015.755147
2018-10-23    22015.765646
2018-10-24    22015.776008
2018-10-25    22015.786236
2018-10-26    22015.796330
Freq: D, Name: predicted_mean, Length: 558, dtype: float64

In [67]:
mse = ((forecast-test['cum_power'] ) ** 2).mean()
mse

7176025.947662519